In [1]:
import os,json
import pandas as pd
import seaborn as sn
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torch.utils.data import DataLoader,Dataset,Subset
from importlib import import_module
from IPython.display import display
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything,LightningDataModule,loggers,seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint
from pl_bolts.callbacks import PrintTableMetricsCallback
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
from dataclasses import dataclass,asdict
seed_everything(42)

/root/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/py38/lib/python3.8/site-packages/pl_bolts/utils/semi_supervised.py:15: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("sklearn", pypi_name="scikit-learn")
/root/miniconda3/envs/py38/lib/python3.8/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaran

42

In [131]:
@dataclass
class Config:
    # custom config
    name:str='test'
    sup_size:int=128
    unsup_size:int=128
    eval_size:int=1000
    dataset:str='cifar10'
    model:str='resnet1D'
    arch:str='fixmatch'
    label_ratio:float=0.1
    min_epochs:int=50
    max_epochs:int=150
    lr:float=3e-4
    threshold:float=0.95
    semi:bool=True
    
    # lightning config
    num_workers:int=8
    auto_scale_batch_size:str=None
    auto_lr_find:bool=False
    default_root_dir:str='/autodl-tmp/checkpoints'
    log_dir:str='/autodl-tmp/logs'
    dataset_dir:str='/autodl-tmp/datasets'
    check_val_every_n_epochs:int=1
    
conf = Config()
vars(conf)

{'name': 'test',
 'sup_size': 128,
 'unsup_size': 128,
 'eval_size': 1000,
 'dataset': 'cifar10',
 'model': 'resnet1D',
 'arch': 'fixmatch',
 'label_ratio': 0.1,
 'min_epochs': 50,
 'max_epochs': 150,
 'lr': 0.0003,
 'threshold': 0.95,
 'semi': True,
 'num_workers': 8,
 'auto_scale_batch_size': None,
 'auto_lr_find': False,
 'default_root_dir': '/autodl-tmp/checkpoints',
 'log_dir': '/autodl-tmp/logs',
 'dataset_dir': '/autodl-tmp/datasets',
 'check_val_every_n_epochs': 1}

In [2]:
from omegaconf import OmegaConf
conf=OmegaConf.load('conf/config.yaml')
conf

{'name': 'test', 'sup_size': 1024, 'unsup_size': 1024, 'eval_size': 1000, 'dataset': 'cls10', 'model': 'resnet1d', 'arch': 'fixmatch', 'label_ratio': 0.1, 'min_epochs': 10, 'max_epochs': 20, 'lr': 0.1, 'semi': True, 'num_workers': 1}

In [26]:
class EMA():
    @staticmethod
    def copy(model:nn.Module,ema:nn.Module):
        ema.load_state_dict(model.state_dict())
        for name, param in model.named_parameters():
            ema[name]=param.data.clone().detach_()
    
    @staticmethod
    def update(model,ema_model,decay,global_step):
        alpha = min(1 - 1 / (global_step +1), decay)
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(alpha).add_(1-alpha, param.data)

class BaseModel(LightningModule):
    def __init__(self,model,ema,methods,*args,**kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.model=model
        self.ema=ema
        EMA.copy(self.model,self.ema)
        self.methods=methods
        
    def forward(self,x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss
    
    def on_train_batch_end(self, outputs, batch, batch_idx):
        EMA.update(self.model,self.ema,0.97,self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=3e-4)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        ema_pred = self.ema(x)
        val_loss = F.cross_entropy(pred, y)
        ema_loss = F.cross_entropy(ema_pred, y)
        self.log_dict({"val_loss": val_loss, "ema_loss": ema_loss})
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        ema_pred = self.ema(x)
        test_loss = F.cross_entropy(pred, y)
        ema_loss = F.cross_entropy(ema_pred, y)
        self.log_dict({"test_loss": test_loss, "ema_test_loss": ema_loss})
        
    def predict_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        return pred

In [119]:
from utils.image_aug import RandAugmentMC
from utils.dataset import TransformWeakStrong as wstwice,TransformBaseWeakStrong as bwstwice
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader,Dataset,Subset
    
from torch.utils.data import DataLoader,random_split
import torchvision as tv

import torchvision.transforms as transforms
class MyDataset(Dataset):
    def __init__(self, data,targets,transfom):
        self.data=data
        self.targets=targets
        self.transfom=transfom
            
    def __getitem__(self, index):
        x,y=self.data[index],self.targets[index]
        _x=self.transfom(x) if self.transfom else x
        return _x,y

    def __len__(self):
        return len(self.data)

class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)


channel_stats = dict(mean = [0.4914, 0.4822, 0.4465],
                            std = [0.2023, 0.1994, 0.2010])
eval_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(**channel_stats)
        ])

class BaseDataset(LightningDataModule):
    def __init__(self,conf:Config) -> None:
        super().__init__()
        self.conf=conf
        self.num_workers=8
        
    
    def setup(self):
        trainset_all = tv.datasets.CIFAR100(root=self.conf.dataset_dir, train=True, download=True)
        testset = tv.datasets.CIFAR100(root=self.conf.dataset_dir, train=False, download=True,transform=eval_transform)
        self.trainset,self.valset=random_split(trainset_all, [0.9,0.1])
        self.testset=testset

    
    def train_dataloader(self):
        weak = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Pad(2, padding_mode='reflect'),
            transforms.RandomCrop(32),
            transforms.ToTensor(),
            transforms.Normalize(**channel_stats)
        ])
        strong = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Pad(2, padding_mode='reflect'),
            transforms.RandomCrop(32),
            RandAugmentMC(n=2, m=10),
            transforms.ToTensor(),
            transforms.Normalize(**channel_stats)
        ])
        
        unsup_transform = wstwice(weak, strong)
        # split sup/unsup from trainset
        _supset,_unsupset=random_split(self.trainset, [0.1,0.9])
        supset=TransformSubset(_supset,weak)
        unsupset=TransformSubset(_unsupset,unsup_transform)

        # return dataloader
        sup_loader=DataLoader(supset, batch_size=self.conf.sup_size, num_workers=self.num_workers, shuffle=True)
        unsup_loader=DataLoader(unsupset, batch_size=self.conf.unsup_size, num_workers=self.num_workers, shuffle=True)
        return [sup_loader,unsup_loader]

    def val_dataloader(self):
        return DataLoader(self.valset, batch_size=self.conf.eval_size, num_workers=self.num_workers, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.testset, batch_size=self.conf.eval_size, num_workers=self.num_workers, shuffle=False)
    
trainset_all = tv.datasets.CIFAR100(root=conf.dataset_dir, train=True, download=True)

Files already downloaded and verified


In [121]:
torch.Tensor().float

/home/lvhang/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/data/dataset.py:342: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [123]:
len(_supset)

50000

In [118]:
@dataclass
class TrainerConfig:
    logger= True
    enable_checkpointing= True
    callbacks= None
    default_root_dir= None
    gradient_clip_val= None
    gradient_clip_algorithm= None
    num_nodes: int = 1
    num_processes= None,  # TODO: Remove in 2.
    devices= None
    gpus= None,  # TODO: Remove in 2.
    auto_select_gpus: bool = False
    tpu_cores= None,  # TODO: Remove in 2.
    ipus= None,  # TODO: Remove in 2.
    enable_progress_bar: bool = True
    overfit_batches= 0.0
    track_grad_norm= -1
    check_val_every_n_epoch= 1
    fast_dev_run= False
    accumulate_grad_batches= None
    max_epochs= None
    min_epochs= None
    max_steps: int = -1
    min_steps= None
    max_time= None
    limit_train_batches= None
    limit_val_batches= None
    limit_test_batches= None
    limit_predict_batches= None
    val_check_interval= None
    log_every_n_steps: int = 50
    accelerator= None
    strategy= None
    sync_batchnorm: bool = False
    precision= 32
    enable_model_summary: bool = True
    num_sanity_val_steps: int = 2
    resume_from_checkpoint= None
    profiler= None
    benchmark= None
    deterministic= None
    reload_dataloaders_every_n_epochs: int = 0
    auto_lr_find= False
    replace_sampler_ddp: bool = True
    detect_anomaly: bool = False
    auto_scale_batch_size= False
    plugins= None
    amp_backend: str = "native"
    amp_level= None
    move_metrics_to_cpu: bool = False
    multiple_trainloader_mode: str = "max_size_cycle"
    inference_mode: bool = True
    
tc=TrainerConfig()

vars(tc)
asdict(tc)

{'num_nodes': 1,
 'auto_select_gpus': False,
 'enable_progress_bar': True,
 'max_steps': -1,
 'log_every_n_steps': 50,
 'sync_batchnorm': False,
 'enable_model_summary': True,
 'num_sanity_val_steps': 2,
 'reload_dataloaders_every_n_epochs': 0,
 'replace_sampler_ddp': True,
 'detect_anomaly': False,
 'amp_backend': 'native',
 'move_metrics_to_cpu': False,
 'multiple_trainloader_mode': 'max_size_cycle',
 'inference_mode': True}

In [ ]:
def main(conf:Config):
    
    callback_t=TQDMProgressBar()
    callback_e=EarlyStopping(monitor='val_loss', min_delta=0.0, patience=5, verbose=False, mode='min', strict=True)
    callback_p=PrintTableMetricsCallback()
    callback_c=ModelCheckpoint(monitor='val_loss', save_top_k=1, save_last=True, save_weights_only=False, mode='min', dirpath=conf.default_root_dir, filename=conf.name+'{epoch:02d}-{val_loss:.2f}')
    tb_logger = loggers.TensorBoardLogger(
        save_dir=conf.log_dir,
        name=conf.name
    )
    callbacks=[callback_e,callback_p,callback_c,callback_t]

    dataset=import_module('datasets.'+conf.dataset)
    model=import_module('models.'+conf.model)
    arch=import_module('arch.'+conf.arch).Arch(model=model,**vars(conf))
    trainer = Trainer(logger=tb_logger,callbacks=callbacks,**vars(conf))
    trainer.fit(arch, dataset)
    
    
    

In [37]:
from utils.image_aug import RandAugmentMC
RandAugmentMC

utils.image_aug.RandAugmentMC

In [2]:
a,b=torch.rand((5,10)),torch.rand((5,10))
a.shape,b.shape

(torch.Size([5, 10]), torch.Size([5, 10]))

In [5]:
trainset=tv.datasets.CIFAR100('/root/autodl-tmp/datasets')

In [58]:
from collections import defaultdict


def uniform_split(dataset,length):
    if isinstance(dataset,torch.utils.data.dataset.Subset):
        # if is subset, restore subset to full-dataset while keep targets stay subset
        indice=np.array(dataset.indices)
        while isinstance(dataset,torch.utils.data.dataset.Subset):
            dataset=dataset.dataset
        targets=np.array(dataset.targets)[indice]
        
    data,targets=dataset.data,np.array(dataset.targets)
    subsets=[]
    unique_labels=np.unique(targets)
    used_label_num=defaultdict(int)
    for ratio in length:
        # for every item in length, construct a subset
        indice=[]
        for y in unique_labels:
            # for every label, append ratio*len idxes into indice
            idxes=np.where(targets==y)[0]
            cur=int(ratio*len(idxes))
            used=used_label_num[y]
            indice.extend(idxes[used:used+cur])
            used_label_num[y]+=cur
        subsets.append(Subset(trainset, indice))
    return subsets
a,b=uniform_split(trainset,[0.9,0.1])

In [84]:
full_targets,indices=np.array(a.dataset.targets),np.array(a.indices)
sub_targets=full_targets[indices]
unique_labels=np.unique(sub_targets)
subsets=[]
used_label_num=defaultdict(int)
for ratio in [0.9,0.1]:
    # for every item in length, construct a subset
    indice=[]
    for y in unique_labels:
        indice_idxes=np.where(sub_targets==y)[0]
        targets_idxes=indices[indice_idxes]
        cur=int(ratio*len(indice_idxes))
        used=used_label_num[y]
        indice.extend(targets_idxes[used:used+cur])
        used_label_num[y]+=cur
    subsets.append(Subset(trainset, indice))
subsets

In [88]:
from collections import Counter


q,w=subsets
i=q.indices
d=np.array(q.dataset.targets)[i]
Counter(d)


Counter({0: 405,
         1: 405,
         2: 405,
         3: 405,
         4: 405,
         5: 405,
         6: 405,
         7: 405,
         8: 405,
         9: 405,
         10: 405,
         11: 405,
         12: 405,
         13: 405,
         14: 405,
         15: 405,
         16: 405,
         17: 405,
         18: 405,
         19: 405,
         20: 405,
         21: 405,
         22: 405,
         23: 405,
         24: 405,
         25: 405,
         26: 405,
         27: 405,
         28: 405,
         29: 405,
         30: 405,
         31: 405,
         32: 405,
         33: 405,
         34: 405,
         35: 405,
         36: 405,
         37: 405,
         38: 405,
         39: 405,
         40: 405,
         41: 405,
         42: 405,
         43: 405,
         44: 405,
         45: 405,
         46: 405,
         47: 405,
         48: 405,
         49: 405,
         50: 405,
         51: 405,
         52: 405,
         53: 405,
         54: 405,
         55: 405,
  

In [57]:
cls50 = ['_R_P_SP','_C_L','_L','_C','_P_SP','_R_SP_N','_C_P','_P_W','_R_P','_P','_C_P_SP','_R_P_SP_N','_R','_C_SP_N','_W','_R_SP','_W_SP','_C_SP','_R_L','_W_N_A','_C_N','_P_A','_SP_N','_R_N','_W_A','_N','_P_SP_A','_P_SP_N_A','_N_A','_P_N','_V_N','_P_W_A','_P_W_N_A','_W_SP_N_A','_CA','_SP_N_A','_R_TEN','_C_A','_TEN','_R_V','_W_CA','_P_W_SP_N_A','_C_R','_P_W_SP_A','_S','_W_S','_P_N_A','_M_P','_M_SP','_V_P']
cls10=['_R_P_SP', '_C_L', '_L', '_P_SP', '_R_SP_N', '_C_P', '_P_W', '_P','_C_P_SP', '_R_P_SP_N']

cls_label={
    'cls10':cls10,
    'cls50':cls50
}

def read_line(line,cls_num):
    if not line:
        return
    try:
        data=json.loads(line)
    except Exception as e:
        return
    labels=cls_label[cls_num]
    label,d=list(data.items())[0]
    if label not in labels:
        return
    _y=labels.index(label)
    _x=[d['i'],d['r'],d['a']]
    return _x,_y
    
def extract_json(filename,cls_num):
    x,y=[],[]
    with open(filename,encoding='UTF-8') as f:
        for line in f.readlines():
            ret=read_line(line,cls_num)
            if not ret:
                continue
            x.append(ret[0])
            y.append(ret[1])
    return x,y

def save_to_npy(x,y,filename):
    np.save(filename+'_x.npy',x)
    np.save(filename+'_y.npy',y)
    
def read_npy(filename):
    x=np.load(filename+'_x.npy')
    y=np.load(filename+'_y.npy')
    return x,y

x,y=extract_json('/root/autodl-nas/test.json','cls10')
save_to_npy(x,y,'/root/autodl-nas/cls10_test')

In [55]:
from collections import Counter
Counter(y)

Counter({1: 9481,
         9: 2186,
         6: 3154,
         3: 6641,
         8: 2310,
         7: 2692,
         4: 4435,
         5: 3773,
         2: 9408,
         0: 5662})

In [56]:
save_to_npy(x,y,'/root/autodl-nas/cls10_train')

In [ ]:
1

: 

: 

In [25]:
p0,p1,p2=torch.randn((100,10)),torch.randn((100,10)),torch.randn((100,10))
h0,h1,h2 = F.softmax(p0,dim=1),F.softmax(p1,dim=1),F.softmax(p2,dim=1)
prob0,label0=torch.max(h0,dim=1)
prob1,label1=torch.max(h1,dim=1)
prob2,label2=torch.max(h2,dim=1)

sh=torch.zeros((3,10))
shc=torch.zeros((3,10))

In [26]:
for idx,prob,label in zip([0,1,2],[prob0,prob1,prob2],[label0,label1,label2]):
    for i in range(10):
        mask=torch.where(label==i)[0]
        sh[idx,i]+=prob[mask].sum()
        shc[idx,i]+=len(mask)

In [45]:
sh,shc

(tensor([[2.2687, 2.8858, 3.4718, 3.3235, 2.2586, 2.7701, 3.1534, 3.3735, 2.3487,
          5.4121],
         [3.8822, 2.7633, 3.9745, 2.9440, 1.8768, 3.7684, 3.8995, 2.4050, 3.8203,
          2.4709],
         [2.6963, 2.9775, 3.6299, 4.4795, 2.6984, 3.4844, 2.8621, 2.6690, 3.5187,
          2.5244]]),
 tensor([[ 7., 10., 11., 11.,  7.,  9., 10., 11.,  9., 15.],
         [12.,  8., 12.,  9.,  7., 11., 12.,  9., 12.,  8.],
         [ 9.,  9., 11., 14.,  8., 11.,  9.,  9., 12.,  8.]]))

In [46]:
sh/shc

tensor([[0.3241, 0.2886, 0.3156, 0.3021, 0.3227, 0.3078, 0.3153, 0.3067, 0.2610,
         0.3608],
        [0.3235, 0.3454, 0.3312, 0.3271, 0.2681, 0.3426, 0.3250, 0.2672, 0.3184,
         0.3089],
        [0.2996, 0.3308, 0.3300, 0.3200, 0.3373, 0.3168, 0.3180, 0.2966, 0.2932,
         0.3156]])

In [56]:
prob,label=torch.randn(10)+(sh/shc)[0],torch.randint(0,10,(10,))
prob,label,(sh/shc)[0]

(tensor([-1.1398,  0.9709,  0.9804,  1.6643, -0.2535, -0.5178,  1.0996,  0.2675,
          1.5618, -1.4261]),
 tensor([8, 4, 1, 9, 3, 1, 8, 3, 9, 7]),
 tensor([0.3241, 0.2886, 0.3156, 0.3021, 0.3227, 0.3078, 0.3153, 0.3067, 0.2610,
         0.3608]))

In [76]:
# 从tensor中随机抽取n个元素
def random_sample(tensor,n):
    idx=torch.randperm(tensor.size(0))[:n]
    return tensor[idx]  

mask1 tensor([False, False, False, False, False, False, False, False, False, False]) cls_prob tensor([])
mask2 tensor([ True, False, False, False,  True,  True, False,  True, False,  True]) m1*m2 tensor([False, False, False, False, False, False, False, False, False, False])
mask1 tensor([False, False,  True, False, False,  True, False, False, False, False]) cls_prob tensor([ 0.9804, -0.5178])
mask2 tensor([ True, False, False, False,  True,  True, False,  True, False,  True]) m1*m2 tensor([False, False, False, False, False,  True, False, False, False, False])
mask1 tensor([False, False, False, False, False, False, False, False, False, False]) cls_prob tensor([])
mask2 tensor([ True, False, False, False,  True,  True, False,  True, False,  True]) m1*m2 tensor([False, False, False, False, False, False, False, False, False, False])
mask1 tensor([False, False, False, False,  True, False, False,  True, False, False]) cls_prob tensor([-0.2535,  0.2675])
mask2 tensor([ True, False, False, Fal

tensor([0.0000, 0.9709, 0.9804, 1.6643, 0.0000, 0.0000, 1.0996, 0.0000, 1.5618,
        0.0000])

In [80]:
import torchvision.transforms as transforms
class ToTensor(object):
    def __init__(self):
        pass

    def __call__(self, origin):
        return torch.Tensor(origin).float()

t=transforms.Compose([
            ToTensor(),
            ToTensor(),
            ToTensor()
        ])

In [83]:
tt=t.transforms

In [85]:
transforms.Compose(tt),t

(Compose(
 ),
 Compose(
 ))

In [87]:
tt[0](np.array([1,2,3]))

tensor([1., 2., 3.])

In [112]:
h=torch.zeros((3,10))/torch.zeros((3,10))
h

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [123]:
h[h.isnan()]

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan])

In [121]:
h[0][0]==torch.float(torch.nan)

TypeError: 'torch.dtype' object is not callable